# ZS DATA-A-THON PHASE 2

### Reading the Data and Initial analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
train_df=pd.read_csv("train.csv")
demog_df=pd.read_csv("demog.csv")
sample_sub=pd.read_csv("submission.csv")

In [2]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37186 entries, 0 to 37185
Data columns (total 12 columns):
HCP_ID           37186 non-null int64
RL               32337 non-null float64
P2P              8050 non-null float64
OLV              20159 non-null float64
RR               27538 non-null float64
DRT              32694 non-null float64
DMS              32807 non-null float64
OLA              26016 non-null float64
DEM              26146 non-null float64
Region           37186 non-null object
Value            37186 non-null object
Speciality_ID    37186 non-null int64
dtypes: float64(8), int64(2), object(2)
memory usage: 3.4+ MB


In [3]:
demog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46485 entries, 0 to 46484
Data columns (total 3 columns):
HCP_ID    46485 non-null int64
gender    46454 non-null object
Age       46485 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [4]:
demog_df.isnull().sum()

HCP_ID     0
gender    31
Age        0
dtype: int64

## Dropping any row in the demog.csv file which is null 

In [5]:
demog_df=demog_df.dropna(how="any")

In [6]:
demog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46454 entries, 0 to 46484
Data columns (total 3 columns):
HCP_ID    46454 non-null int64
gender    46454 non-null object
Age       46454 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [7]:
demog_df.head(10)

,HCP_ID,gender,Age
0,14586,FEMALE,34
1,46157,MALE,34
2,16600,MALE,34
3,16601,MALE,33
4,16602,FEMALE,33
5,16603,MALE,32
6,16604,FEMALE,32
7,16605,FEMALE,31
8,16606,MALE,31
9,16607,MALE,31


In [8]:
train_df.head(10)

,HCP_ID,RL,P2P,OLV,RR,DRT,DMS,OLA,DEM,Region,Value,Speciality_ID
0,9304,0.645833,0.475755,NaN,NaN,0.000,0.169355,NaN,0.004458,urban,H,307999
1,9305,0.419355,0.190302,NaN,0.115167,0.000,0.012195,NaN,0.000000,urban,H,307999
2,9306,1.000000,NaN,NaN,0.134362,0.000,0.149254,0.129584,0.113179,urban,L,307999
3,9307,0.564102,NaN,0.054703,0.076778,NaN,0.138889,0.158047,0.002574,urban,H,307999
4,9308,0.584906,NaN,0.145874,0.383891,0.000,0.198198,0.125850,0.044009,urban,H,307999
5,9309,0.611111,NaN,NaN,0.076778,0.000,0.052632,0.135165,0.000000,urban,M,307999
6,9310,0.666667,NaN,0.018234,0.191945,0.000,NaN,0.109352,0.000000,urban,H,307999
7,9311,0.529411,NaN,NaN,0.134362,0.000,0.062500,0.217449,0.015967,urban,M,307999
8,9312,0.657895,NaN,0.036468,0.115167,0.000,0.171875,NaN,NaN,urban,H,307999
9,9313,0.612903,0.095151,NaN,0.345502,0.175,0.047872,0.189591,0.000000,urban,H,307999


In [9]:
demog_df.describe()

,HCP_ID,Age
count,46454.000000,46454.000000
mean,23258.405390,81.934193
std,13429.513748,32.948621
min,1.000000,22.000000
25%,11622.250000,52.000000
50%,23264.500000,89.000000
75%,34887.750000,110.000000
max,46513.000000,121.000000


# Merging the 2 files with the column HCP_ID as the primary key

In [10]:
combined_df=pd.merge(train_df,demog_df,how='left',on='HCP_ID')
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37186 entries, 0 to 37185
Data columns (total 14 columns):
HCP_ID           37186 non-null int64
RL               32337 non-null float64
P2P              8050 non-null float64
OLV              20159 non-null float64
RR               27538 non-null float64
DRT              32694 non-null float64
DMS              32807 non-null float64
OLA              26016 non-null float64
DEM              26146 non-null float64
Region           37186 non-null object
Value            37186 non-null object
Speciality_ID    37186 non-null int64
gender           37158 non-null object
Age              37158 non-null float64
dtypes: float64(9), int64(2), object(3)
memory usage: 4.3+ MB


## Creating Dummy columns then binarizing them for better model fitting

In [11]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df
for i in ["Region","Speciality_ID","Value","gender"]:
    combined_df = create_dummies(combined_df,i)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37186 entries, 0 to 37185
Data columns (total 38 columns):
HCP_ID                  37186 non-null int64
RL                      32337 non-null float64
P2P                     8050 non-null float64
OLV                     20159 non-null float64
RR                      27538 non-null float64
DRT                     32694 non-null float64
DMS                     32807 non-null float64
OLA                     26016 non-null float64
DEM                     26146 non-null float64
Region                  37186 non-null object
Value                   37186 non-null object
Speciality_ID           37186 non-null int64
gender                  37158 non-null object
Age                     37158 non-null float64
Region_rural            37186 non-null uint8
Region_urban            37186 non-null uint8
Speciality_ID_18766     37186 non-null uint8
Speciality_ID_92938     37186 non-null uint8
Speciality_ID_96835     37186 non-null uint8
Speciality_ID_11

In [12]:
combined_df.head()

,HCP_ID,RL,P2P,OLV,RR,DRT,DMS,OLA,DEM,Region,...,Speciality_ID_371651,Speciality_ID_462171,Speciality_ID_483435,Speciality_ID_569454,Value_H,Value_L,Value_M,Value_U,gender_FEMALE,gender_MALE
0,9304,0.645833,0.475755,NaN,NaN,0.0,0.169355,NaN,0.004458,urban,...,0,0,0,0,1,0,0,0,0,1
1,9305,0.419355,0.190302,NaN,0.115167,0.0,0.012195,NaN,0.000000,urban,...,0,0,0,0,1,0,0,0,1,0
2,9306,1.000000,NaN,NaN,0.134362,0.0,0.149254,0.129584,0.113179,urban,...,0,0,0,0,0,1,0,0,0,1
3,9307,0.564102,NaN,0.054703,0.076778,NaN,0.138889,0.158047,0.002574,urban,...,0,0,0,0,1,0,0,0,0,1
4,9308,0.584906,NaN,0.145874,0.383891,0.0,0.198198,0.125850,0.044009,urban,...,0,0,0,0,1,0,0,0,1,0


In [13]:
temp=combined_df
temp=temp.drop(["Region","Speciality_ID","Value","gender"],axis=1)

In [14]:
temp.describe()

,HCP_ID,RL,P2P,OLV,RR,DRT,DMS,OLA,DEM,Age,...,Speciality_ID_371651,Speciality_ID_462171,Speciality_ID_483435,Speciality_ID_569454,Value_H,Value_L,Value_M,Value_U,gender_FEMALE,gender_MALE
count,37186.000000,32337.000000,8050.000000,20159.000000,27538.000000,32694.000000,32807.000000,26016.000000,26146.000000,37158.000000,...,37186.000000,37186.000000,37186.000000,37186.000000,37186.000000,37186.000000,37186.000000,37186.000000,37186.000000,37186.000000
mean,27910.118297,0.601944,0.189566,0.155577,0.221124,0.036384,0.118864,0.192901,0.072392,81.876339,...,0.003066,0.013311,0.022320,0.048217,0.635185,0.026865,0.334266,0.003684,0.266902,0.732346
std,10741.107350,0.184662,0.179546,0.191303,0.197190,0.095047,0.174413,0.113425,0.088854,32.954994,...,0.055284,0.114606,0.147725,0.214227,0.481385,0.161691,0.471739,0.060586,0.442346,0.442742
min,9304.000000,0.022727,0.024893,0.017238,0.002376,0.000000,0.000000,0.000000,0.000000,22.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18612.250000,0.491228,0.068771,0.034476,0.068164,0.000000,0.010101,0.109769,0.000000,51.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27910.500000,0.630435,0.137542,0.086190,0.163704,0.000000,0.058642,0.185038,0.046951,89.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,37210.750000,0.740000,0.208524,0.189618,0.313554,0.022222,0.145299,0.263858,0.104650,110.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,46513.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
li=["DMS","DRT","RL","RR","DEM","OLA","OLV","P2P"]
temp["Age"].replace(np.NaN,temp["Age"].mean(),inplace=True)
temp.isnull().sum()

HCP_ID                      0
RL                       4849
P2P                     29136
OLV                     17027
RR                       9648
DRT                      4492
DMS                      4379
OLA                     11170
DEM                     11040
Age                         0
Region_rural                0
Region_urban                0
Speciality_ID_18766         0
Speciality_ID_92938         0
Speciality_ID_96835         0
Speciality_ID_117101        0
Speciality_ID_155443        0
Speciality_ID_262386        0
Speciality_ID_295159        0
Speciality_ID_306909        0
Speciality_ID_307999        0
Speciality_ID_321702        0
Speciality_ID_333055        0
Speciality_ID_334432        0
Speciality_ID_371651        0
Speciality_ID_462171        0
Speciality_ID_483435        0
Speciality_ID_569454        0
Value_H                     0
Value_L                     0
Value_M                     0
Value_U                     0
gender_FEMALE               0
gender_MAL

## Categorizing Age column to prevent Biasing

In [16]:
def process_age(df):
    """Process the Age column into pre-defined 'bins' 

    Usage
    ------

    train = process_age(train)
    """
    cut_points = [0,5,12,18,35,60,125]
    label_names = ["Infant","Child","Teenager","Young Adult","Adult","Senior"]
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)
    return df
temp=process_age(temp)
print(temp.shape)
temp = create_dummies(temp,"Age_categories")
temp.shape

(37186, 35)


(37186, 41)

In [17]:
temp=temp.drop(["Age","Age_categories"],axis=1)
temp.columns

Index(['HCP_ID', 'RL', 'P2P', 'OLV', 'RR', 'DRT', 'DMS', 'OLA', 'DEM',
       'Region_rural', 'Region_urban', 'Speciality_ID_18766',
       'Speciality_ID_92938', 'Speciality_ID_96835', 'Speciality_ID_117101',
       'Speciality_ID_155443', 'Speciality_ID_262386', 'Speciality_ID_295159',
       'Speciality_ID_306909', 'Speciality_ID_307999', 'Speciality_ID_321702',
       'Speciality_ID_333055', 'Speciality_ID_334432', 'Speciality_ID_371651',
       'Speciality_ID_462171', 'Speciality_ID_483435', 'Speciality_ID_569454',
       'Value_H', 'Value_L', 'Value_M', 'Value_U', 'gender_FEMALE',
       'gender_MALE', 'Age_categories_Infant', 'Age_categories_Child',
       'Age_categories_Teenager', 'Age_categories_Young Adult',
       'Age_categories_Adult', 'Age_categories_Senior'],
      dtype='object')

## Using GridSearchCV for finding the best model for Prediction

In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
import xgboost

def select_model(df,features,target):
    
    all_X = df[features]
    all_y = df[target]

    # List of dictionaries, each containing a model name,
    # it's estimator and a dict of hyperparameters
    models = [
        {
            "name": "LinearRegression",
            "estimator": LinearRegression(),
            "hyperparameters":
                {
                    "fit_intercept": [True, False],
                    "copy_X": [True, False],
                    "n_jobs": [1,2]
                }
        },
        {
            "name": "RandomForestRegressor",
            "estimator": RandomForestRegressor(random_state=1),
            "hyperparameters":
                {
                    "n_estimators": [10],
                    "max_depth": [12],
                    "max_features": ["auto"],
                    "min_samples_leaf": [2],
                    "min_samples_split": [2]

                }
        },
        {
            "name": "XGBoost",
            "estimator": xgboost.XGBRegressor(  gamma=0, subsample=0.75,
                           colsample_bytree=1),
            "hyperparameters":
                {
                    "n_estimators": [20,40],
                    "learning_rate":[0.08,0.1],
                    "max_depth": [12]
                }
        }
    ]

    for model in models:
        print(model['name'])
        print('-'*len(model['name']))

        grid = GridSearchCV(model["estimator"],
                            param_grid=model["hyperparameters"],
                            cv=10)
        grid.fit(all_X,all_y)
        model["best_params"] = grid.best_params_
        model["best_score"] = grid.best_score_
        model["best_model"] = grid.best_estimator_

        print("Best Score: {}".format(model["best_score"]))
        print("Best Parameters: {}\n".format(model["best_params"]))

    return models

scorer=make_scorer(mean_squared_error,False)
def check(model,train,test):
    n_folds=5
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(train.values)
    smape=cross_val_score(model,train,test,scoring=scorer,cv=kf)
    smape=smape**0.5
    print('mean: ',np.mean(smape),'\n')
    print('median: ',np.median(smape),'\n')
    print('all: ',smape)

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
li=("DMS","DRT","RL","RR","DEM","OLA","OLV","P2P")
import random

random.seed(6)
temp=temp.sort_values("HCP_ID")
def select_features(df,target):
    # Remove non-numeric columns, columns that have null values
    #df = df.select_dtypes([np.number]).dropna(axis=1)
    #df=df.dropna(how="any")
    all_X = df.drop(target,axis=1)
    all_y = df[target]
    
    clf = RandomForestRegressor(random_state=1)
    selector = RFECV(clf,cv=2)
    selector.fit(all_X,all_y)
    
    best_columns = list(all_X.columns[selector.support_])
    
    return best_columns
result_df=pd.DataFrame()
result_df["HCP_ID"]=temp["HCP_ID"]
for i in li:
    temp_li=list(li)
    tempp=temp.copy()
    temp_li.remove(i)
    for j in temp_li:
        tempp[j].replace(np.NaN,tempp[j].mean(),inplace=True)
        
    #sepereting the dataset
    train=tempp[tempp[i].notnull()]
    test=tempp[tempp[i].isnull()].drop(i,axis=1)
    
    cols=select_features(train,i)
    #select_model(train,cols,i)
    
    #model
    model=RandomForestRegressor(random_state=2,n_estimators= 10,max_depth=12,max_features="auto",
                                min_samples_leaf=2,min_samples_split=2)
    model.fit(train[cols],train[i])
    pred=model.predict(test[cols])
    
    #putting the column back
    test[i]=pred
    frames=[train,test]
    result=pd.concat(frames)
    resultt=result[["HCP_ID",i]]
    result_df=pd.merge(result_df,resultt,how='left',on='HCP_ID').sort_values("HCP_ID")
    temp=temp.drop(i,axis=1)
    temp[i]=result_df[i]

## The result DataFrame

In [20]:
result_df.shape

(37186, 9)

In [21]:
result_df.head()

,HCP_ID,DMS,DRT,RL,RR,DEM,OLA,OLV,P2P
0,9304,0.169355,0.000000,0.645833,0.187506,0.004458,0.127568,0.071773,0.475755
1,9305,0.012195,0.000000,0.419355,0.115167,0.000000,0.100478,0.074792,0.190302
2,9306,0.149254,0.000000,1.000000,0.134362,0.113179,0.129584,0.177508,0.255306
3,9307,0.138889,0.014407,0.564102,0.076778,0.002574,0.158047,0.054703,0.132769
4,9308,0.198198,0.000000,0.584906,0.383891,0.044009,0.125850,0.145874,0.202111


# Submission file

In [22]:
result_df.to_csv("SUBMISSION_PHASE_2.csv",index=False)